In [1]:
from qem.fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import os
# sys.path.append('/home/zzhang/OneDrive/code/hyperspy/hyperspy')
%matplotlib qt

file = '/home/zzhang/OneDrive/code/qem/data/high_entropy/adf_avg.npy'
# s = hs.load(file)
image = np.load(file)
dx = 0.36254824166962796

model=ImageModelFitting(image, dx=dx,units='A')

# get the unique coordinates by row
# input_coordinates = np.unique(input_coordinates, axis=0)

In [2]:
model.find_peaks(min_distance=4)

INFO:root:Removing peak at (675.0, 252.0).
INFO:root:Removing peak at (695.0, 254.0).
INFO:root:Removing peak at (717.0, 245.0).
INFO:root:Removing peak at (662.0, 250.0).
INFO:root:Removing peak at (657.0, 253.0).
INFO:root:Adding peak at (674.6276330219717, 121.48797413304086) with atom type 1.
INFO:root:Adding peak at (670.2710925386681, 141.63697386832052) with atom type 12.


Updated peak locations.


array([[152.        , 185.        ],
       [152.        , 216.        ],
       [151.        ,  92.        ],
       ...,
       [713.        , 161.        ],
       [674.62763302, 121.48797413],
       [670.27109254, 141.63697387]])

In [2]:
model.find_peaks(min_distance=4)
cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
model.map_lattice(cif_file=cif_file,elements=['Sr','Ti'],min_distance=20,a_limit=50, b_limit=50 )
# model.select_region(invert_selection=False)

Updated peak locations.
Origin selected: [275. 170.]
Vector a selected: [12.  0.]
Vector b selected: [  0. -10.]
In pixel: Origin: [275. 170.], Vector a: [12.  0.], Vector b: [  0. -10.]
In space: Origin: [99.70076646 61.63320108] A, Vector a: [4.3505789 0.       ] A, Vector b: [ 0.         -3.62548242] A
Origin selected: [360. 128.]
Vector a selected: [65. -1.]
Vector b selected: [ 3. 25.]
In pixel: Origin: [360. 128.], Vector a: [65. -1.], Vector b: [ 3. 25.]
In space: Origin: [3.87879415 1.37912681] 1/A, Vector a: [ 0.70033783 -0.01077443] 1/A, Vector b: [0.03232328 0.26936071] 1/A
FFT real a: [11.05622332 -0.47839428] pixel, Real b: [ 0.43611275 10.22139252] pixel


In [5]:
model.select_region()

INFO:root:Selected 2713 atoms out of 3176 atoms.


In [6]:

model.fit_background = True
params = model.init_params()
model.fit_random_batch(params, batch_size=1000, maxiter=50, tol=1e-3, plot=False, step_size=0.01,verbose=False,num_epoch=5)

Fitting random batch:   0%|          | 0/3 [00:00<?, ?it/s]INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
Fitting random batch: 100%|██████████| 3/3 [00:06<00:00,  2.13s/it]
INFO:root:Convergence rate for pos_x = 0.5906253177801091
INFO:root:Convergence rate for pos_y = 0.5837708947974569
INFO:root:Convergence rate for height = 7.941320567333316e-05
INFO:root:Convergence rate for sigma = 0.0044049604538272434
INFO:root:Convergence not reached
Fitting random batch: 100%|██████████| 3/3 [00:03<00:00,  1.09s/it]
INFO:root:Convergence rate for pos_x = 0.5687255859375
INFO:root:Convergence rate for pos_y = 0.5694828033447266
INFO:root:Convergence rate for height = 7.513328671664758e-05
INFO:root:Convergence rate fo

{'pos_x': array([  5.16073227,   3.50016427,   3.99327898, ..., 631.52124023,
        631.29718018, 633.03723145]),
 'pos_y': array([  3.6004734 ,  14.76122761,  26.81199837, ..., 178.14056396,
        123.57289124,  57.48521042]),
 'height': array([3120.02913123, 2886.78847547, 3182.20434441, ..., 2151.82403418,
        1916.49524455, 1294.39872316]),
 'sigma': array([1.88387735, 1.88387735, 1.88387735, ..., 1.88387735, 1.88387735,
        1.93331316]),
 'background': 31493.609731826564}

In [3]:
from qem.crystal_analyzer import CrystalAnalyzer

crystal_analyzer = CrystalAnalyzer(image = model.image, dx = model.dx, peak_positions = model.coordinates, atom_types = model.atom_types, elements=['Sr','Ti'],units=model.units)
cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
crystal_analyzer.read_cif(cif_file)
crystal_analyzer.choose_lattice_vectors(tolerance=20)
supercell_in_image, supercell_atom_types = crystal_analyzer.generate_supercell_lattice(a_limit=20, b_limit=20)

Origin selected: [343. 189.]
Vector a selected: [10. -1.]
Vector b selected: [ -1. -11.]
In pixel: Origin: [343. 189.], Vector a: [10. -1.], Vector b: [ -1. -11.]
In space: Origin: [124.35404689  68.52161768] A, Vector a: [ 3.62548242 -0.36254824] A, Vector b: [-0.36254824 -3.98803066] A
Origin selected: [360. 128.]
Vector a selected: [65. -1.]
Vector b selected: [ 3. 25.]
In pixel: Origin: [360. 128.], Vector a: [65. -1.], Vector b: [ 3. 25.]
In space: Origin: [3.87879415 1.37912681] 1/A, Vector a: [ 0.70033783 -0.01077443] 1/A, Vector b: [0.03232328 0.26936071] 1/A
FFT real a: [11.05622332 -0.47839428] pixel, Real b: [ 0.43611275 10.22139252] pixel


In [4]:
peaks = crystal_analyzer.shift_origin_adaptive(20,20)

plt.imshow(model.image)
for key, value in peaks.items():
    plt.scatter(value[0], value[1])

In [7]:
model.plot_scs()

In [12]:
model.plot_fitting()

In [7]:
model.plot_coordinates()

In [1]:
from qem.crystal_analyzer import CrystalAnalyzer
from qem.fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

file = '/home/zzhang/OneDrive/code/qem/data/STO/adf_average_STO.txt'
image = np.loadtxt(file)
dx= 0.1645429228960236	
model=ImageModelFitting(image, dx=dx,units='A')
model.find_peaks()
crystal_analyzer = CrystalAnalyzer(image = model.image, dx = model.dx, peak_positions = model.coordinates, atom_types = model.atom_types, elements=['Sr','Ti'])

INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Updated peak locations.


INFO:root:No atoms selected.


In [6]:

cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
crystal_analyzer.read_cif(cif_file)

Atoms(symbols='SrTi', pbc=True, cell=[3.905, 3.905, 3.905], spacegroup_kinds=...)

In [4]:
crystal_analyzer.choose_lattice_vectors()

Selection incomplete.
Selection incomplete.


TypeError: unsupported operand type(s) for *: 'NoneType' and 'NoneType'

In [5]:
crystal_analyzer._origin_adaptive = None
shift_center = crystal_analyzer.shift_origin_adaptive(50, 50)
plt.imshow(model.image, cmap='gray')
peaks = []
for key, value in shift_center.items():
    peaks.append(value)

plt.scatter(np.array(peaks)[:,0], np.array(peaks)[:,1], c='r', s=10)
plt.show()

In [6]:
peak_positions, atom_types = crystal_analyzer.generate_supercell_lattice(a_limit=50, b_limit=50)

In [14]:
peak_positions, atom_types = crystal_analyzer.supercell_project_2d()

In [7]:
plt.scatter(peak_positions[:,0], peak_positions[:,1], c='r', s=10)

In [10]:
crystal_analyzer.unitcell_mapping()

TypeError: list indices must be integers or slices, not numpy.float64

In [23]:
im_refined = model.predict_local(params=model.params)
plt.imshow(im_refined)

In [25]:
plt.imshow(model.image-im_refined)

In [18]:
model.atom_types

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,